In [2]:
import Pkg; Pkg.add("QuantumCumulants")
import Pkg; Pkg.add("Random"); Pkg.add("Distributions")
using Random, Distributions
using QuantumCumulants, OrdinaryDiffEq, ModelingToolkit, BenchmarkTools, Plots

    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
function gauss(x, x0, σ)
    return exp(-((x-x0)/σ)^2)
end
detuning = LinRange(0,1,4)

4-element LinRange{Float64, Int64}:
 0.0,0.333333,0.666667,1.0

In [4]:
function cumu_solve(N, Δ_a, kappa, Fs)
    # N: number of spins
    # Δ_a: detuning of each spin, 1D array of N
    @cnumbers Δ_c F g γ κ
    # Define hilbert space
    hf = FockSpace(:cavity)
    ha = [NLevelSpace(Symbol(:atom,i),2) for i=1:N]
    h = ⊗(hf, ha...)
    # Define the fundamental operators
    a = Destroy(h,:a)
    σe(i,j,k) = Transition(h,Symbol(:σe, k),i,j,k+1)
    # Hamiltonian
    H = Δ_c*a'*a + F*(a+a') + 0.5*sum(Δ_a[i]*(σe(2,2,i)-σe(1,1,i)) for i=1:N) + g*sum(a'*σe(1,2,i) + a*σe(2,1,i) for i=1:N)
    # Collapse operators
    J = [a;[σe(1,2,i) for i=1:N]]
    rates = [κ;[γ for i=1:N]]
    # Derive equations for populations
    ops = [σe(2,2,i) for i=1:N]
    sz_ops = [(σe(2,2,i)-σe(1,1,i)) for i=1:N]
    eqs = meanfield(ops,H,J;rates=rates,order=2)
    # Complete but neglect phase-dependent terms
    ϕ(x::Average) = ϕ(x.arguments[1])
    ϕ(::Destroy) = -1
    ϕ(::Create) = 1
    ϕ(x::QTerm) = sum(map(ϕ, x.args_nc))
    ϕ(x::Transition) = x.i - x.j
    phase_invariant(x) = iszero(ϕ(x))
    # complete!(eqs;filter_func=phase_invariant)
    complete!(eqs)
    @named sys = ODESystem(eqs)
    u0 = zeros(ComplexF64, length(eqs))
    u0[1:N] .= 0.0 # atoms are inverted initially
    p0 = (Δ_c=>0, F=>Fs, g=>1.6, γ=>0, κ=>kappa)
    prob = ODEProblem(sys,u0,(0.0,1),p0)
    sol = solve(prob,RK4())
    n_pho = sqrt.(abs2.(sol[a'*a]))
    # n_sz = Array{Float64, 2}(undef, N, length(sol.t))
    # for i = 1:N
    #     n_sz[i] = sqrt.(abs2.(sol[sz_ops[i]]))
    # end
    pe = [real.(sol[σe(2,2,i)]) for i = 1:N]
    # plot(sol.t, n, xlabel="t", label="n")"
    return sol.t, n_pho, eqs
end


cumu_solve (generic function with 1 method)

In [1]:
sol, n_pho, eqs = cumu_solve(20, [0 for i=1:20], 20, 8)
plot(sol, n_pho)

UndefVarError: UndefVarError: cumu_solve not defined